In [1]:
import django
import os
import json
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
django.setup()

In [2]:
import pandas as pd
import requests
from io import BytesIO

In [3]:
from data_management.models import *

In [4]:
CroppedImage.objects.select_related('origin_source')

<QuerySet [<CroppedImage: CroppedImage object (178)>, <CroppedImage: CroppedImage object (179)>, <CroppedImage: CroppedImage object (180)>, <CroppedImage: CroppedImage object (181)>, <CroppedImage: CroppedImage object (182)>, <CroppedImage: CroppedImage object (183)>, <CroppedImage: CroppedImage object (184)>, <CroppedImage: CroppedImage object (185)>, <CroppedImage: CroppedImage object (186)>, <CroppedImage: CroppedImage object (187)>, <CroppedImage: CroppedImage object (188)>, <CroppedImage: CroppedImage object (189)>, <CroppedImage: CroppedImage object (190)>, <CroppedImage: CroppedImage object (191)>, <CroppedImage: CroppedImage object (192)>, <CroppedImage: CroppedImage object (193)>, <CroppedImage: CroppedImage object (194)>, <CroppedImage: CroppedImage object (195)>, <CroppedImage: CroppedImage object (196)>, <CroppedImage: CroppedImage object (197)>, '...(remaining elements truncated)...']>

In [7]:
CroppedImage.objects.filter(origin_source__image__isnull=False).first().origin_source.image.url

'https://monde-data.s3.amazonaws.com/original-bag-images-dev/009KR55482K_1.jpg'

In [24]:
left = []
top = []
right = []
bottom = []
origin_url = []
width = []
height = []

In [25]:
def get_image_size(resp):
    from PIL import Image
    byteImgIO = BytesIO()
    byteImg = Image.open(BytesIO(resp.content))
    byteImg.save(byteImgIO, "JPEG")
    byteImgIO.seek(0)
    byteImg = byteImgIO.read()
    dataBytesIO = BytesIO(byteImg)
    image = Image.open(dataBytesIO)
    image = image.convert('RGB')
    width, height = image.size
    return width, height

In [38]:
a={}
a['a']=[1]
print(a)
a['a'] = a['a'].append(2)
print(a)

{'a': [1]}
{'a': None}


In [28]:
for c in CroppedImage.objects.all()[:10]:
    if c.origin_source.image:
        url = c.origin_source.image.url
        origin_url.append(url)
        resp = requests.get(url)
        w, h = get_image_size(resp)
        width.append(w)
        height.append(h)
        left.append(c.left*w)
        top.append(c.top*h)
        right.append(c.right*w)
        bottom.append(c.bottom*h)
    elif c.origin_source.s3_image_url:
        url = c.origin_source.s3_image_url
        origin_url.append(url)
        resp = requests.get(url)
        w, h = get_image_size(resp)
        width.append(w)
        height.append(h)
        left.append(c.left*w)
        top.append(c.top*h)
        right.append(c.right*w)
        bottom.append(c.bottom*h)
    else:
        origin_url.append("")
        width.append("")
        height.append("")

In [37]:
dic = {}
dic['left']=left
print(dic['left'])
dic['top']=top
dic['right']=right
dic['bottom']=bottom
dic['origin_url']=origin_url
dic['width']=width
dic['height']=height
print(len(dic['left']))

[Decimal('0.1088'), Decimal('0.1234'), Decimal('0.2437'), Decimal('0.0175'), Decimal('0.0267'), Decimal('0.0304'), Decimal('0.0307'), Decimal('0.0231'), Decimal('0.0131'), Decimal('0.0592')]
10


In [31]:
test_data_pd=pd.DataFrame.from_dict(dic)

In [32]:
test_data_pd

,left,top,right,bottom,origin_url,width,height
0,0.1088,0.0736,0.9796,0.9332,https://monde-data.s3.amazonaws.com/original-b...,2352,2352
1,0.1234,0.0137,0.8547,0.9970,https://monde-data.s3.amazonaws.com/original-b...,627,471
2,0.2437,0.1885,0.8038,0.9606,https://monde-data.s3.amazonaws.com/original-b...,309,309
3,0.0175,0.2182,0.9817,0.7890,https://monde-data.s3.amazonaws.com/original-b...,2352,2352
4,0.0267,0.0102,0.9762,0.9702,https://monde-data.s3.amazonaws.com/original-b...,2352,2352
5,0.0304,0.2482,0.9654,0.7661,https://monde-data.s3.amazonaws.com/original-b...,2352,2352
6,0.0307,0.4273,0.9474,0.9519,https://monde-data.s3.amazonaws.com/original-b...,447,559
7,0.0231,0.1217,0.9788,0.9030,https://monde-data.s3.amazonaws.com/original-b...,1019,1317
8,0.0131,0.0329,1.0000,0.9465,https://monde-data.s3.amazonaws.com/original-b...,299,375
9,0.0592,0.4290,0.9167,0.9273,https://monde-data.s3.amazonaws.com/original-b...,447,559


In [58]:
test_data_pd.to_csv("test_pd.csv", mode='w', index=False)